In [1]:
# import csv

# orderers = []
# summary = []

# with open('D:\\me\\masters\\data\\google_books_1299.csv', encoding='utf-8') as file:
#     data = csv.reader(file)
#     for k in data:
#         print(k)

In [25]:
import csv 
import json
from bson.objectid import ObjectId

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
    unusedColumns = ['','rating','voters','published_date','page_count', 'generes', 'currency']
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf)   

        #convert each csv row into python dict
        for row in csvReader: 
            row['price'] = round(float(row['price']) * 0.27, 2)
            row['genres'] = [element.strip() for element in list(row['generes'].split(','))]
            row['pageCount'] = int(row['page_count'])
            row['publishedDate'] = row['published_date']
            row['imageUrl'] = 'https://mastersimages.blob.core.windows.net/images/book.jpg'

            for column in unusedColumns:
                del row[column]
            
            #add this python dict to json array
            jsonArray.append(row)

    # print(jsonArray)

    result = list({v['title']:v for v in jsonArray}.values())
    print(len(result))
  
    # convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(result, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'D:\\me\\university\\masters\\data\\google_books_1299.csv'
jsonFilePath = r'D:\\me\\university\\masters\\data\\books.json'
csv_to_json(csvFilePath, jsonFilePath)

246


In [1]:
import nest_asyncio
nest_asyncio.apply()

In [2]:
import motor.motor_asyncio
from config import *

client = motor.motor_asyncio.AsyncIOMotorClient(connectionString)
db = client.test

try:
    print(client.server_info())
except Exception:
    print('Unable to connect to the server.')

db = client['masters']
collection = db.books
collection

ModuleNotFoundError: No module named 'pymongo.mongo_replica_set_client'

In [3]:
from pymongo import MongoClient
from config import *

client = MongoClient()
client = MongoClient(connectionString)
db = client['masters']
collection = db.books
collection

Collection(Database(MongoClient(host=['masters-shard-00-02.cpq5u.mongodb.net:27017', 'masters-shard-00-00.cpq5u.mongodb.net:27017', 'masters-shard-00-01.cpq5u.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-wrhudr-shard-0', tls=True), 'masters'), 'books')

In [4]:
import asyncio

async def do_find_one(client):
    db = client['masters']
    collection = db.books
    # collection
    document = await collection.find_one({'price': {'$lt': 12}})
    print(document)

loop = asyncio.get_event_loop()
loop.run_until_complete(do_find_one(client))

TypeError: object dict can't be used in 'await' expression

In [5]:
from bson.objectid import ObjectId

async def do_insert():
    result = await db.books.insert_many(
        [{ '_id': ObjectId(),'i': i} for i in range(3)])

loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert())

In [34]:
def readJsonFile(fileName):
    with open(fileName, encoding='utf-8') as file: 
        # print(json.load(file))
        return json.load(file)

async def do_insert(dataset, fileName):
    data = readJsonFile(fileName)
    result = await db[dataset].insert_many(
        [ i for i in data])

loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert('books', 'D:\\me\\masters\\data\\books.json'))

<coroutine object do_insert at 0x0000029750D527A0>

In [16]:
loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert('users', 'D:\\me\\masters\\data\\users.json'))

In [5]:
import asyncio

async def do_delete_many():
    coll = db.books
    n = await coll.count_documents({})
    print('%s documents before calling delete_many()' % n)
    result = await db.books.delete_many({'i': {'$ne': ''}})
    print('%s documents after' % (await coll.count_documents({})))

loop = asyncio.get_event_loop()
loop.run_until_complete(do_delete_many())

RuntimeError: This event loop is already running

In [ ]:
import asyncio

async def do_delete_many():
    coll = db.rates
    n = await coll.count_documents({})
    print('%s documents before calling delete_many()' % n)
    result = await db.rates.delete_many({'i': {'$ne': ''}})
    print('%s documents after' % (await coll.count_documents({})))

loop = asyncio.get_event_loop()
loop.run_until_complete(do_delete_many())

In [7]:
async def do_find(dataset):
    result = []
    cursor = db[dataset].find({})
    for document in await cursor.to_list(length=1400):
        # print(document)
        result.append(document)
    return result

# loop = asyncio.get_event_loop()
# loop.run_until_complete(do_find())

In [10]:
import random
import datetime

loop = asyncio.get_event_loop()
books = loop.run_until_complete(do_find('books'))[0:10]
# books = do_find('books')

loop = asyncio.get_event_loop()
users = loop.run_until_complete(do_find('users'))[0:10]
# users = do_find('users')

rates = []

for i in range(10):
    dict = {}
    dict['bookId'] = books[i]['_id']
    dict_list = []

    for j in range(10):
        usersRandom = random.randint(0, 9)        

        rate = {}
        rate['userId'] = users[usersRandom]['_id']
        rate['rate'] = random.randint(1, 5)
        rate['comment'] = ''
        day_diff = random.randint(0, 3)
        yesterday = datetime.datetime.utcnow() - datetime.timedelta(days = day_diff)
        rate['createdAt'] = datetime.datetime(yesterday.year, yesterday.month, yesterday.day)

        existing = False
        
        # print(dict_list)
        for item in dict_list:
            # print(item)
            if(item['userId'] == rate['userId']):
                existing = True

        if(existing == False):
            dict_list.append(rate)

    dict['reviews'] = dict_list
    rates.append(dict)

# print(rates)

async def do_insert_data(data):
    result = await db.rates.insert_many(
        [ i for i in data])
    
loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert_data(rates))

In [33]:
from dateutil import tz
today = datetime.datetime.utcnow().date()
start = datetime.datetime(today.year, today.month, today.day)
start

datetime.datetime(2022, 4, 27, 0, 0)

In [39]:
yesterday_datetime = datetime.datetime.utcnow() - datetime.timedelta(days = 1)
yesterday = datetime.datetime(yesterday_datetime.year, yesterday_datetime.month, yesterday_datetime.day)
today = yesterday + datetime.timedelta(days = 1)
today

datetime.datetime(2022, 4, 27, 0, 0)

In [12]:
date = Date()
type(date)

NameError: name 'Date' is not defined

In [39]:
orders = []

for i in range(300):
    booksAmount = random.randint(1, 5)
    usersRandom = random.randint(0, 119)

    dict = {}
    orderBooks = []
    
    dict['userId'] = users[usersRandom]['_id']

    for i in range(booksAmount):
        booksRandom = random.randint(0, 245)
        orderBooks.append(books[booksRandom]['_id'])

    dict['booksId'] = orderBooks

    orders.append(dict)
    # print(dict)


print(len(orders))

async def do_insert_data(data):
    result = await db.orders.insert_many(
        [ i for i in data])
    
loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert_data(orders))

{'userId': ObjectId('620bb9fef23b1bc78052c651'), 'booksId': [ObjectId('620e9cb61feee707fc548d47'), ObjectId('620e9cb61feee707fc548d9e')]}
{'userId': ObjectId('620bb9fef23b1bc78052c5fb'), 'booksId': [ObjectId('620e9cb61feee707fc548d2a'), ObjectId('620e9cb61feee707fc548d59')]}
{'userId': ObjectId('620bb9fef23b1bc78052c63f'), 'booksId': [ObjectId('620e9cb61feee707fc548d7e'), ObjectId('620e9cb61feee707fc548d5a')]}
{'userId': ObjectId('620bb9fef23b1bc78052c5f2'), 'booksId': [ObjectId('620e9cb61feee707fc548d49'), ObjectId('620e9cb61feee707fc548d1b'), ObjectId('620e9cb61feee707fc548db3'), ObjectId('620e9cb61feee707fc548dd1'), ObjectId('620e9cb61feee707fc548dc0')]}
{'userId': ObjectId('620bb9fef23b1bc78052c618'), 'booksId': [ObjectId('620e9cb61feee707fc548d64')]}
{'userId': ObjectId('620bb9fef23b1bc78052c608'), 'booksId': [ObjectId('620e9cb61feee707fc548dc6'), ObjectId('620e9cb61feee707fc548dae')]}
{'userId': ObjectId('620bb9fef23b1bc78052c60f'), 'booksId': [ObjectId('620e9cb61feee707fc548d25'

In [1]:
import datetime
str = datetime.datetime.utcnow().strftime('%d.%m.%Y %H:%M:%S')
print(str)

datetime.datetime.strptime(str, '%d.%m.%Y %H:%M:%S')

23.02.2022 16:06:01


'23.02.2022 16:06:01'

In [15]:
datetime.datetime.utcnow() < (datetime.datetime.utcnow() - datetime.timedelta(days = 1))

False

In [13]:
(datetime.datetime.utcnow() - datetime.timedelta(days = 1)).strftime('%Y-%m-%d-%H:%M:%S')

'2022-02-21-18:09:51'

In [7]:
from bson.objectid import ObjectId
a = ObjectId()
a.__str__()

'62154d12e642272bf9189970'

In [5]:
import re

txt = datetime.datetime.utcnow().strftime('%d.%m.%Y %H:%M:%S')
x = re.search('^23.02.2022 ', txt)
x

<re.Match object; span=(0, 11), match='23.02.2022 '>

In [30]:
import json
import datetime

def readJsonFile(fileName):
    with open(fileName, encoding='utf-8') as file: 
        # print(json.load(file))
        return json.load(file)

data = readJsonFile('D:\\me\\university\\masters\\data\\books.json')

authors = []
publishers = []
genres = []

for record in data:
    date = record['publishedDate']
    record['publishedDate'] = datetime.datetime.strptime(date, '%b %d, %Y')

    record_genres_list = record['genres']

    record_genres_dict_list = []
    for item in record_genres_list:
        if item.endswith(' &amp'):
            item = item.replace(' &amp', '')
        genre_exist = False
        for existing_gerne in genres:
            if existing_gerne['name'] == item:
                record_genres_dict_list.append(existing_gerne)
                genre_exist = True
                break
        if genre_exist == False:
            new_gerne = {'_id': ObjectId(), 'name': item}
            genres.append(new_gerne)
            record_genres_dict_list.append(new_gerne)
    
    record['genres'] = record_genres_dict_list

    author_exist = False
    for author in authors:
        if author['name'] == record['author']:
            author_exist = True
            record['author'] = author
            break
    if author_exist == False:
        new_author = {'_id': ObjectId(), 'name': record['author']}
        authors.append(new_author)
        record['author'] = new_author

    publisher_exist = False
    for publisher in publishers:
        if publisher['name'] == record['publisher']:
            publisher_exist = True
            record['publisher'] = publisher
            break
    if author_exist == False:
        new_publisher = {'_id': ObjectId(), 'name': record['publisher']}
        publishers.append(new_publisher)
        record['publisher'] = new_publisher

print(len(publishers))

182


In [ ]:
def readJsonFile(fileName):
    with open(fileName, encoding='utf-8') as file: 
        # print(json.load(file))
        return json.load(file)

def preprocess_data(data):
    authors = []
    publishers = []
    genres = []

    for record in data:
        date = record['publishedDate']
        record['publishedDate'] = datetime.datetime.strptime(date, '%b %d, %Y')

        record_genres_list = record['genres']

        record_genres_dict_list = []
        for item in record_genres_list:
            if item.endswith(' &amp'):
                item = item.replace(' &amp', '')
            genre_exist = False
            for existing_gerne in genres:
                if existing_gerne['name'] == item:
                    record_genres_dict_list.append(existing_gerne)
                    genre_exist = True
                    break
            if genre_exist == False:
                new_gerne = {'_id': ObjectId(), 'name': item}
                genres.append(new_gerne)
                record_genres_dict_list.append(new_gerne)
        
        record['genres'] = record_genres_dict_list

        author_exist = False
        for author in authors:
            if author['name'] == record['author']:
                author_exist = True
                record['author'] = author
                break
        if author_exist == False:
            new_author = {'_id': ObjectId(), 'name': record['author']}
            authors.append(new_author)
            record['author'] = new_author

        publisher_exist = False
        for publisher in publishers:
            if publisher['name'] == record['publisher']:
                publisher_exist = True
                record['publisher'] = publisher
                break
        if author_exist == False:
            new_publisher = {'_id': ObjectId(), 'name': record['publisher']}
            publishers.append(new_publisher)
            record['publisher'] = new_publisher

    return data, authors, publishers, genres

async def do_insert(dataset, data):
    result = await db[dataset].insert_many(
        [ i for i in data])

data = readJsonFile('D:\\me\\masters\\data\\books.json')
preprocessed_data = preprocess_data(data)

loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert('books', preprocessed_data[0]))

loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert('authors', preprocessed_data[1]))

loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert('publishers', preprocessed_data[2]))

loop = asyncio.get_event_loop()
loop.run_until_complete(do_insert('genres', preprocessed_data[3]))

In [23]:
a = [{'id':1, 'name':'1'},
{'id':2, 'name':'2'},
{'id':3, 'name':'3'}]

if '2' in a:
    print('s')

In [9]:
a = ObjectId()
a

ObjectId('6305e9f4649cbc5e7d82e937')

In [19]:
a = ['Cognitive Psychology &amp', 'Something']
for i in a:
    if i.endswith(' &amp'):
        i = i.replace(' &amp', '')
        print(i)
a

Cognitive Psychology


['Cognitive Psychology &amp', 'Something']

In [18]:
db['books'].drop()
db['authors'].drop()
db['publishers'].drop()
db['genres'].drop()

In [19]:
import json
import datetime
from bson.objectid import ObjectId

def readJsonFile(fileName):
    with open(fileName, encoding='utf-8') as file: 
        # print(json.load(file))
        return json.load(file)

def preprocess_data(data):
    authors = []
    publishers = []
    genres = []

    for record in data:
        date = record['publishedDate']
        record['publishedDate'] = datetime.datetime.strptime(date, '%b %d, %Y')

        record_genres_list = record['genres']

        record_genres_dict_list = []
        for item in record_genres_list:
            if item.endswith(' &amp'):
                item = item.replace(' &amp', '')
            genre_exist = False
            for existing_gerne in genres:
                if existing_gerne['name'] == item:
                    record_genres_dict_list.append(existing_gerne)
                    genre_exist = True
                    break
            if genre_exist == False:
                new_gerne = {'_id': ObjectId(), 'name': item}
                genres.append(new_gerne)
                record_genres_dict_list.append(new_gerne)
        
        record['genres'] = record_genres_dict_list

        author_exist = False
        for author in authors:
            if author['name'] == record['author']:
                author_exist = True
                record['author'] = author
                break
        if author_exist == False:
            new_author = {'_id': ObjectId(), 'name': record['author']}
            authors.append(new_author)
            record['author'] = new_author

        publisher_exist = False
        for publisher in publishers:
            if publisher['name'] == record['publisher']:
                publisher_exist = True
                record['publisher'] = publisher
                break
        if publisher_exist == False:
            new_publisher = {'_id': ObjectId(), 'name': record['publisher']}
            publishers.append(new_publisher)
            record['publisher'] = new_publisher

    return data, authors, publishers, genres

data = readJsonFile('D:\\me\\university\\masters\\data\\books.json')
preprocessed_data = preprocess_data(data)

db.books.insert_many(preprocessed_data[0])
db.authors.insert_many(preprocessed_data[1])
db.publishers.insert_many(preprocessed_data[2])
db.genres.insert_many(preprocessed_data[3])

In [19]:
import json
import datetime
from bson.objectid import ObjectId
import random

books = db.books.find()[0:10]

users = db.users.find()[0:10]

rates = []

for i in range(10):
    dict = {}
    dict['bookId'] = books[i]['_id']
    dict_list = []

    for j in range(10):
        usersRandom = random.randint(0, 9)        

        rate = {}
        rate['user'] = {'_id': users[usersRandom]['_id'], 'name': users[usersRandom]['name']}
        rate['rate'] = random.randint(1, 5)
        rate['comment'] = 'Lorem ipsum dolor sit amet..'
        day_diff = random.randint(0, 3)
        yesterday = datetime.datetime.utcnow() - datetime.timedelta(days = day_diff)
        rate['createdAt'] = datetime.datetime(yesterday.year, yesterday.month, yesterday.day)

        existing = False
        
        # print(rate)
        for item in dict_list:
            # print(item)
            if(item['user']['_id'] == rate['user']['_id']):
                existing = True

        if(existing == False):
            dict_list.append(rate)

    dict['reviews'] = dict_list
    rates.append(dict)

print(rates)

# db['rates'].drop()
db.rates.insert_many(rates)

[{'bookId': ObjectId('630602ac073e94de49556747'), 'reviews': [{'user': {'_id': ObjectId('620bb9fef23b1bc78052c5ec'), 'name': 'Deanne Evans'}, 'rate': 1, 'comment': 'Lorem ipsum dolor sit amet..', 'createdAt': datetime.datetime(2022, 9, 8, 0, 0)}, {'user': {'_id': ObjectId('620bb9fef23b1bc78052c5f1'), 'name': 'Lakisha Tucker'}, 'rate': 5, 'comment': 'Lorem ipsum dolor sit amet..', 'createdAt': datetime.datetime(2022, 9, 8, 0, 0)}, {'user': {'_id': ObjectId('620bb9fef23b1bc78052c5f2'), 'name': 'Jordan Slater'}, 'rate': 3, 'comment': 'Lorem ipsum dolor sit amet..', 'createdAt': datetime.datetime(2022, 9, 9, 0, 0)}, {'user': {'_id': ObjectId('620bb9fef23b1bc78052c64b'), 'name': 'Rosalind Fuller'}, 'rate': 4, 'comment': 'Lorem ipsum dolor sit amet..', 'createdAt': datetime.datetime(2022, 9, 9, 0, 0)}, {'user': {'_id': ObjectId('620bb9fef23b1bc78052c64e'), 'name': 'Juarez Marquez'}, 'rate': 2, 'comment': 'Lorem ipsum dolor sit amet..', 'createdAt': datetime.datetime(2022, 9, 9, 0, 0)}, {'use

In [17]:
db['rates'].drop()

In [2]:
a = [{"book1":"book11",
"book2":"book21",
"rate":1},
{"book1":"book12",
"book2":"book22",
"rate":1},
{"book1":"book13",
"book2":"book23",
"rate":1}]

In [4]:
any("book11" in item.values() and "book11" in item.values() for item in a)

True